## Test Neuroevolución

In [48]:
%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
import pandas as pd
from sklearn.preprocessing import StandardScaler  

In [49]:
from deap import algorithms
from deap import base
from deap import creator
from deap import tools
import random
import pickle
from copy import deepcopy

Importación del scaler
http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html

### Cargo los datos y retiro el Timestamp y el score

In [50]:
df = pd.read_csv("gameStates.csv", sep=',', header=None)
df.columns = ['timeStamp','Px', 'Py', 'heat', 'Exp1','Eyp1','Exp2','Eyp2', 'Ex1', 'Ey1', 'Eh1', 'Ex2', 'Ey2', 'Eh2', 'Ex3', 'Ey3', 'Eh3', 'Ex4', 'Ey4','Eh4','Ex5', 'Ey5','Eh5','Ex6', 'Ey6','Eh6','ray1','ray2','ray3','ray4','ray5','ray6','ray7','ray8','ray9','ray10','ray11','ray12','ray13','ray14','ray15','ray16','ray17','ray18','ray19','ray20','ray21','ray22','ray23','ray24','ray25','ray26','ray27','score',"VKey","HKey","Shooting"]
df.drop(['timeStamp','score', 'Ex1', 'Ey1', 'Eh1', 'Ex2', 'Ey2', 'Eh2', 'Ex3', 'Ey3', 'Eh3', 'Ex4', 'Ey4','Eh4','Ex5', 'Ey5','Eh5','Ex6', 'Ey6','Eh6'],axis=1,inplace=True)
df = df[1:6001]

#### Transforma el porblema multioputput en multilabel

In [51]:
def multioutput2multilabel(row):
    values = pd.Series([(row["VKey"] == "UpArrow"),(row["VKey"] == "DownArrow"),(row["HKey"] == "LeftArrow"),(row["HKey"] == "RightArrow"),row["Shooting"]])
    return pd.concat([row[:len(row)-3], values.astype(float)])

df = df.apply(multioutput2multilabel,axis=1)

In [52]:
#Defino los elementos train_data y target_data
train = df.drop([0, 1, 2,3,4],axis=1).values
labels = df[[0,1,2,3,4]].values

In [53]:
#define el perceptrón y lo entrena
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(15,), random_state=1)
clf.fit(train, labels)
model = deepcopy(clf)
pickle.dump(clf, open(".\\AI_Data\\clf.sav", 'wb'))
pickle.dump(model, open(".\\AI_Data\\model.sav", 'wb'))
shapes = [coef.shape for coef in model.coefs_] # solo voy a ajustar los pesos
sizes =[coef.size for coef in model.coefs_]

In [41]:
#Neuroevolución

def gen2Coefs(gen,sizes,shapes):
    coefs = []
    splits = np.split(gen, [sizes[0]])
    for i in range(len(splits)):
        coefs.append(splits[i].reshape(shapes[i]))
    return coefs

def coefs2gen(coefs,sizes,shapes):
    return np.concatenate((coefs[0].flatten(),coefs[1].flatten()))




In [57]:
def MLPFitness(individual):
    coefs=gen2Coefs(np.array(individual),sizes,shapes)
    model.coefs_ = coefs
    pickle.dump(model, open(".\\AI_Data\\model.sav", 'wb'))
    os.system("AI.exe")
    scores = pd.read_csv("myData.csv", sep=',', header=None)
    os.remove("myData.csv")
    return np.mean(scores.values)
    

# Fitness del gen de la red original sobre los datos modificados
MLPFitness(coefs2gen(clf.coefs_,sizes,shapes))

-143.85714285714286

### Pipeline

In [55]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler


estimators = [('stantandarize', StandardScaler()), ('clf', MLPClassifier())]

pipe = Pipeline(estimators)

# entreno el pipe como si fuera un clasificador, 
# también podría hacer validación cruzada y todo como si fuese un clasificador normal
pipe.fit(train, labels)
pickle.dump(model, open(".\\AI_Data\\pipe.sav", 'wb'))

from sklearn.model_selection import cross_val_score
print(cross_val_score(pipe, train, labels, cv=5).mean())

0.184833333333


In [56]:
MLPClassifier()

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [46]:
def MLPFitness(individual):
    coefs=gen2Coefs(np.array(individual),sizes,shapes)
    model.named_steps['clf'].coefs_ = coefs 
    pickle.dump(model, open(".\\AI_Data\\model.sav", 'wb'))
    os.system("AI.exe")
    scores = pd.read_csv("myData.csv", sep=',', header=None)
    os.remove("myData.csv")
    return np.mean(scores.values) 
    


In [ ]:
N = 1

pesosIniciales = coefs2gen(pipe.named_steps['clf'].coefs_,sizes,shapes).copy()
shapes = [coef.shape for coef in model.coefs_] # solo voy a ajustar los pesos
sizes =[coef.size for coef in model.coefs_]


creator.create("FitnessMax", base.Fitness, weights=(1.0,)) # Maximiza
creator.create("Individual", np.ndarray, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

toolbox.register("attr_float", random.random)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=sum(sizes))
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("individual_guess", lambda :creator.Individual(pesosIniciales))
toolbox.register("population_mix",initPopulation,list,toolbox.individual, toolbox.individual_guess)       


toolbox.register("evaluate", MLPFitness)
toolbox.register("mate", tools.cxOnePoint)

toolbox.register("mutate", tools.mutGaussian, mu=0.5, sigma=0.5, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=10)

random.seed(64)
# con 50 ya va    
population = toolbox.population_mix(n=3, n_guess=N)  
    
hof = tools.HallOfFame(1, similar=np.array_equal)
    
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)
    
population, logbook = algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=1, stats=stats, halloffame=hof)
#print(hof) # hall of the fame contiene el mejor individuo vivo en cada generacion
#mejorInd = hof[0] # esta ordenado de manera que el primer elemento es el mejor de siempre

#MLPFitness(mejorInd)

In [43]:
def gen2Coefs(gen,sizes,shapes):
    coefs = []
    splits = np.split(gen, [sizes[0]])
    for i in range(len(splits)):
        coefs.append(splits[i].reshape(shapes[i]))
    return coefs

def coefs2gen(coefs,sizes,shapes):
    return np.concatenate((coefs[0].flatten(),coefs[1].flatten()))

In [44]:
def initPopulation(pop, ind_random, ind_guess,n_guess=1,n=10):
    
    pop = []
    n_random = n - n_guess
    for i in range(n_random):
        pop.append(ind_random())
        
    for i in range(n_guess):
        pop.append(ind_guess())
    
    return pop

In [45]:
N = 1

pesosIniciales = coefs2gen(clf.coefs_,sizes,shapes).copy()
shapes = [coef.shape for coef in model.coefs_] # solo voy a ajustar los pesos
sizes =[coef.size for coef in model.coefs_]


creator.create("FitnessMax", base.Fitness, weights=(1.0,)) # Maximiza
creator.create("Individual", np.ndarray, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

toolbox.register("attr_float", random.random)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=sum(sizes))
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("individual_guess", lambda :creator.Individual(pesosIniciales))
toolbox.register("population_mix",initPopulation,list,toolbox.individual, toolbox.individual_guess)       


toolbox.register("evaluate", MLPFitness)
toolbox.register("mate", tools.cxOnePoint)

toolbox.register("mutate", tools.mutGaussian, mu=0.5, sigma=0.5, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=10)

random.seed(64)
# con 50 ya va    
population = toolbox.population_mix(n=3, n_guess=N)  
    
hof = tools.HallOfFame(1, similar=np.array_equal)
    
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)
    
population, logbook = algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2, ngen=1, stats=stats, halloffame=hof)
#print(hof) # hall of the fame contiene el mejor individuo vivo en cada generacion
#mejorInd = hof[0] # esta ordenado de manera que el primer elemento es el mejor de siempre

#MLPFitness(mejorInd)

FileNotFoundError: File b'myData.csv' does not exist